In [1]:
# !pip install tensorboard==1.15
!nvidia-smi
gpus= "0,1,2,3"

import warnings
warnings.filterwarnings(action='ignore')

import os
os.environ["CUDA_DEVICE_ORDER"]= "PCI_BUS_ID";
os.environ["CUDA_VISIBLE_DEVICES"]= gpus;

import torch
gpu_count = torch.cuda.device_count()
print(torch.cuda.is_available())
print(gpu_count)
print(torch.__version__)

Fri Oct 16 10:26:22 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.100      Driver Version: 440.100      CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  TITAN X (Pascal)    Off  | 00000000:05:00.0 Off |                  N/A |
| 28%   49C    P8    17W / 250W |     10MiB / 12194MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  TITAN X (Pascal)    Off  | 00000000:06:00.0 Off |                  N/A |
| 25%   44C    P8    11W / 250W |     10MiB / 12196MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   2  T

In [2]:
def threholding(inputs,value):
    result = inputs.copy()
    print(np.unique(result))
    result[result<value]=0
    result[result!=value]=1
    return result

In [31]:
import os
import random
from argparse import ArgumentParser, Namespace

import numpy as np
import torch
import torch.nn.functional as F
import torchvision.transforms as transforms
from PIL import Image
from torch.utils.data import DataLoader, Dataset

import pytorch_lightning as pl
from pytorch_lightning.loggers import WandbLogger

class SegModel(pl.LightningModule):
    """
    Semantic Segmentation Module
    This is a basic semantic segmentation module implemented with Lightning.
    It uses CrossEntropyLoss as the default loss function. May be replaced with
    other loss functions as required.
    It is specific to KITTI dataset i.e. dataloaders are for KITTI
    and Normalize transform uses the mean and standard deviation of this dataset.
    It uses the FCN ResNet50 model as an example.
    Adam optimizer is used along with Cosine Annealing learning rate scheduler.
    """

    def __init__(self, **kwargs):
        super().__init__()
        
#         self.batch_size = hparams.batch_size
#         self.lr = hparams.lr
        self.batch_size = batch_size
        self.lr = lr
        self.loss = loss
#         self.save_hyperparameters()  
        self.metric = pl.metrics.F1()
        self.net = net

    def forward(self, x):
        return self.net(x)

    def training_step(self, batch, batch_idx):
        x = batch['data']
        y = batch['seg']
        x = x.float()
        y = y.float()
            
        y_hat = self(x) #deepSup
        loss= self.loss(y_hat,y)
        metric = self.metric(y_hat,y)
        
        if batch_idx==0:
            x = x[0].squeeze().cpu().detach().numpy()*255
            y = y[0].squeeze().cpu().detach().numpy()*255
            y_hat = y_hat[0].squeeze().cpu().detach().numpy().round()*255
            x = np.moveaxis(x,0,-1)

            
#             x = x[0]
#             fig, ax = plt.subplots(figsize=(16, 12))
#             ax.imshow(x)
#             ax.imshow(y,alpha=0.2)
#             neptune_logger.experiment.log_image('train', fig)
            image = np.hstack([x[...,0],y,y_hat])
            neptune_logger.log_image('image_train', image)
#             wandb.log({'Train':[wandb.Image(x, masks={
#                                                     "prediction" : {"mask_data" : pred, "class_labels" : class_labels},
#                                                     "ground truth" : {"mask_data" : y, "class_labels" : class_labels }
#                                                     })]
#                       })
        
        result = pl.TrainResult(loss)
        result.log('train_loss', loss, on_epoch=True)
        result.log('train_acc', metric, on_epoch=True)
        
        return result

    def validation_step(self, batch, batch_idx):
        x = batch['data']
        y = batch['seg']
        x = x.float()
        y = y.float()
        y_hat = self(x) #deepSup        
        loss = self.loss(y_hat,y)
        metric = self.metric(y_hat,y)
        
        if batch_idx==0:
            x = x[0].squeeze().cpu().detach().numpy()*255
            y = y[0].squeeze().cpu().detach().numpy()*255
            y_hat = y_hat[0].squeeze().cpu().detach().numpy().round()*255
            x = np.moveaxis(x,0,-1)
            
            image = np.hstack([x[...,0],y,y_hat])
            neptune_logger.log_image('image_valid', image)
            
#         if batch_idx==0:
#             x = x[0].squeeze().cpu().detach().numpy()
#             y = y[0].squeeze().cpu().detach().numpy()
#             pred = pred[0].squeeze().cpu().detach().numpy().round()
#             x = np.moveaxis(x,0,-1)

#             wandb.log({'Valid':[wandb.Image(x, masks={
#                                                     "prediction" : {"mask_data" : pred, "class_labels" : class_labels},
#                                                     "ground truth" : {"mask_data" : y, "class_labels" :class_labels }
#                       })]
#                       })
            
        result = pl.EvalResult(checkpoint_on=loss)
        result.log('val_loss', loss, on_epoch=True)
        result.log('val_acc', metric, on_epoch=True)
        return result
    
    def test_step(self, batch, batch_idx):
        x = batch['data']
        y = batch['seg']
        fname = batch['fname']
        x = x.float()
        y = y.float()

#         y_hat = self(x) #deepSup        
        
#         from monai.inferers import SlidingWindowInferer
        
#         net = self.net
#         infer = SlidingWindowInferer(roi_size=(512, 512), sw_batch_size=1, overlap=0.75)
#         class ToyModel:
#             pred = 0
#             def __call__(self, input):
#                 input = net(input)
#         return input
    
#         y_hat = infer(x, ToyModel())
#         print(y_hat.shape)
        
        y_hat = pred_tta_seg(net,x,512,100)
        loss = self.loss(y_hat,y)
        metric = self.metric(y_hat,y)
        
        for idx_ in range(len(y_hat)):
            path ="Output/"+fname[idx_].split('/')[-1]
            
            x = x[idx_].cpu().detach().numpy()
            y = y[idx_].cpu().detach().numpy()
            
            # stack y and result
            print(y_hat.shape,np.unique(y_hat))
            output = y_hat[idx_].cpu().detach().numpy().round()
#             output = np.hstack((np.expand_dims(x[2],0),y,output))   # stack 해서 저장하기!
            output = np.moveaxis(output,0,-1)            
            skimage.io.imsave(path,output)
            
        result = pl.EvalResult(checkpoint_on=loss)
        result.log('test_loss', loss)
        result.log('test_acc', metric)
        print(metric)
        return result

    def configure_optimizers(self):
        opt = torch.optim.Adam(self.net.parameters(), lr=self.lr)
        sch = torch.optim.lr_scheduler.CosineAnnealingLR(opt, T_max=10)
        return [opt], [sch]
    
    def train_dataloader(self):
        return DataLoader(train_dataset, shuffle=True, batch_size=self.batch_size) 
    
    def val_dataloader(self):
        return DataLoader(valid_dataset, shuffle=False, batch_size=self.batch_size) 

    def test_dataloader(self):
        return DataLoader(test_dataset, shuffle=False, batch_size=1) 

In [32]:
# !pip install -U git+https://github.com/albu/albumentations
import albumentations as albu

def augmentation_train():
    train_transform = [
        albu.HorizontalFlip(p=0.5),
        albu.VerticalFlip(p=0.5),
        albu.CLAHE(clip_limit=(3,3), tile_grid_size=(4, 4),p=1),
        albu.MultiplicativeNoise(multiplier=(0.9, 1.1), per_channel=True, p=1),
        albu.RandomBrightnessContrast(brightness_limit=(-0.2, 0.2),contrast_limit=(-0.2, 0.2),brightness_by_max=True, p=0.5),
        albu.RandomGamma(gamma_limit=(80,120), p=0.5),
#         albu.RandomShadow(p=0.5),
        
        albu.OneOf([
        albu.ElasticTransform(border_mode=cv2.BORDER_CONSTANT,interpolation=cv2.INTER_LINEAR,alpha=1,sigma=50,alpha_affine=50, p=0.5),
        albu.GridDistortion(border_mode=cv2.BORDER_CONSTANT,interpolation=cv2.INTER_LINEAR,distort_limit=(-0.2,0.2),num_steps=5, p=0.5),
        albu.OpticalDistortion(border_mode=cv2.BORDER_CONSTANT,interpolation=cv2.INTER_LINEAR,distort_limit=(-.05,.05),shift_limit=(-0.05,0.05), p=0.5),
        ]
        ,p=0.5),
        
        albu.OneOf([            
        albu.IAASharpen(alpha=(0,0.3), lightness=(0.01,0.05), p=0.3),
        albu.GaussianBlur(blur_limit=(3), p=0.1),
#         albu.MotionBlur(blur_limit=(3), p=0.1),
        ],p=0.5),
        
#         albu.Resize(height=1024, width=1024, always_apply=True),        
#         albu.ShiftScaleRotate(border_mode=cv2.BORDER_CONSTANT, interpolation=cv2.INTER_LINEAR, shift_limit=0.1, scale_limit=0.2, rotate_limit=10, p=0.5),
        albu.RandomResizedCrop(height=512, width=512, scale=(0.1,0.3),ratio=(.8,1.2), interpolation=cv2.INTER_LINEAR, always_apply=True), 
#         albu.RandomCrop(height=512, width=512, always_apply=True), 

        albu.ToGray(p=1)

    ]
    return albu.Compose(train_transform)

def augmentation_valid(center=None):
    test_transform = [
#         albu.Resize(height=1024, width=1024, always_apply=True),        
#         albu.RandomCrop(height=512, width=512, always_apply=True), 
        albu.CLAHE(clip_limit=(3,3), tile_grid_size=(4, 4),p=1),
        albu.RandomResizedCrop(height=512, width=512, scale=(0.1,0.3),ratio=(.8,1.2), interpolation=cv2.INTER_LINEAR, always_apply=True), 
        albu.ToGray(p=1)
        
    ]
    return albu.Compose(test_transform)

def augmentation_test(center=None):
    test_transform = [
        albu.Resize(height=1024, width=1024, always_apply=True),        
#         albu.RandomCrop(height=512, width=512, always_apply=True), 
        albu.CLAHE(clip_limit=(3,3), tile_grid_size=(4, 4),p=1),
#         albu.RandomResizedCrop(height=512, width=512, scale=(0.1,0.3),ratio=(.8,1.2), interpolation=cv2.INTER_LINEAR, always_apply=True), 
        albu.ToGray(p=1)
        
    ]
    return albu.Compose(test_transform)

import skimage
from skimage.morphology import erosion, dilation, closing, disk
from skimage import exposure
from skimage import morphology
from skimage.measure import label, regionprops

# !sudo pip install --upgrade https://github.com/VincentStimper/mclahe/archive/numpy.zip
# import mclahe as mc

import numpy as np

import matplotlib.pyplot as plt
def image_mchahe_3ch(image,imagetype='numpy'):
    """
    Input : H x W x C or C x H x W image
    """
    result = np.zeros_like(image)
    if imagetype=='numpy':
        result[...,0] = mc.mclahe(image[...,0])
        result[...,1] = mc.mclahe(image[...,1])
        result[...,2] = mc.mclahe(image[...,2])
    else:
        result[0] = mc.mclahe(image[0])
        result[1] = mc.mclahe(image[1])
        result[2] = mc.mclahe(image[2])
    return result

from skimage import data, io, filters
from scipy import ndimage as ndi
from skimage import feature
from skimage.morphology import skeletonize
from skimage.morphology import square
import scipy

class Dataset_npy():#DataLoader):
    def __init__(self, x_list, y_list, augmentation=None):
        self.x_list = x_list
        self.y_list = y_list
        self.augmentation = augmentation
        
    def __len__(self):
        return len(self.x_list)
  
    def __getitem__(self, index):
        try:
            image = np.load(self.x_list[index])
        except:
            image = cv2.imread(self.x_list[index])
#             image = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
#             image = np.expand_dims(image,-1)
            image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
#             image = cv2.cvtColor(image,cv2.COLOR_BGR2HSV)
        try:
            gt_seg = cv2.imread(self.y_list[index],0).astype(np.float32)
        except:
            gt_seg = np.zeros((image.shape[1],image.shape[2])).astype(np.float32)
        
        gt_seg[gt_seg!=0] = 1      
        gt_seg[gt_seg!=1] = 0
        
        gt_seg = np.expand_dims(gt_seg,-1)
        
        if self.augmentation:
            sample = self.augmentation(image = image, mask = gt_seg)
            image, gt_seg = sample['image'], sample['mask']
        
        if np.any(gt_seg):
            gt = np.array([1]).astype(np.float32)
        else:
            gt = np.array([0]).astype(np.float32)

        image = np.moveaxis(image/255,-1,0)
        gt_seg = np.moveaxis(gt_seg,-1,0)
#         gt_seg = np.squeeze(gt_seg).astype(np.long)
        return {"data":image, "seg":gt_seg, "cls":gt, "fname":self.x_list[index]}       
    
# !sudo rm -r Mytorch
# !sudo rm -r Keeplearning

# !sudo git clone https://github.com/kevinkwshin/Mytorch.git
# !sudo git clone https://github.com/kevinkwshin/Keeplearning.git
    
# !sudo pip install -r Mytorch/requirements.txt --quiet

# import Mytorch
# from Mytorch.unet3plus.UNet_3Plus import *
# from Mytorch.__init__ import *

import Keeplearning
from Keeplearning.image_processing import *
from Keeplearning.list_processing import *
from Keeplearning.load_data import *
from Keeplearning.metrics import *
from Keeplearning.visualize import *

import wandb
import cv2
import glob

path_image_hrf='../dataset/retina_hrf/image/'
path_label_hrf='../dataset/retina_hrf/artery_vein_segmentation/'
path_image_asan='../dataset/ASAN/retina_20200921/'
path_label_asan='../dataset/ASAN/labeling_20200921/'

x_list_hrf  = sorted(glob.glob(path_image_hrf+'*'))
x_list_asan = sorted(glob.glob(path_image_asan+'*'))
y_list_hrf = sorted(glob.glob(path_label_hrf+'*'))
y_list_asan = sorted(glob.glob(path_label_asan+'*'))

x_train = x_list_hrf[:35] + x_list_asan[:27] 
y_train = y_list_hrf[:35] + y_list_asan[:27]
x_valid = x_list_hrf[35:42] + x_list_asan[27:32]
y_valid = y_list_hrf[35:42] + y_list_asan[27:32]
x_test = x_list_hrf[42:] + x_list_asan[32:]
y_test = y_list_hrf[42:] + y_list_asan[32:]

path_image='../dataset/ASAN_100_forLabel/'
path_label='../dataset/ASAN_100_forLabel/label/'
x_list=list_sort_nicely(glob.glob(path_image+'*'))
y_list=list_sort_nicely(glob.glob(path_label+'*'))

# path_image='../../retina_for_GAN_fractal_dim/'
# path_label='../../retina_for_GAN_fractal_dimss/'
# x_list=list_sort_nicely(glob.glob(path_image+'*'))
# y_list=list_sort_nicely(glob.glob(path_label+'*'))

x_test = x_list
y_test = y_list

print(len(x_train),len(y_train),len(x_valid),len(y_valid),len(x_test),len(y_test))

batch_size = 1

from torch.utils.data import Dataset, DataLoader

train_dataset = Dataset_npy(x_train,y_train,
                            augmentation=augmentation_train()
                            )

valid_dataset = Dataset_npy(x_valid,y_valid,
                            augmentation=augmentation_valid()
                            )

test_dataset = Dataset_npy(x_test,y_test,
                            augmentation=augmentation_test(),
                            )

train_loader = DataLoader(train_dataset,
                          batch_size=batch_size,
                          num_workers=2,
                          shuffle=True,
                         )

# import pickle
from torch.utils.data import Dataset, DataLoader
batch_size = 1

valid_loader = DataLoader(valid_dataset,
                          batch_size=batch_size,
                          num_workers=0,
#                           shuffle=True,
                         )

test_loader = DataLoader(test_dataset,
                          batch_size=batch_size,
                          num_workers=0,
#                           shuffle=True,
                         )


62 62 12 12 100 0


In [33]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class BoundaryLoss(nn.Module):
    """Boundary Loss proposed in:
    Alexey Bokhovkin et al., Boundary Loss for Remote Sensing Imagery Semantic Segmentation
    https://arxiv.org/abs/1905.07852
    we set θ0 to 3 and θ to 5-7 as a proper choice
    """

    def __init__(self, theta0=3, theta=5, dims=4):
        super().__init__()

        self.theta0 = theta0
        self.theta = theta
        self.dims = dims

    def forward(self, pred, gt):
        """
        Input:
            - pred: the output from model (after softmax)
                    shape (N, C, H, W)
            - gt: ground truth map , float type
                    shape (N, C, H, w)
        Return:
            - boundary loss, averaged over mini-batch
        """
#         n, c, _, _ = pred.shape
        n = pred.shape[0]
        c = pred.shape[1]

        # one-hot vector of ground truth
        one_hot_gt = gt #one_hot(gt, c)

        # boundary map
        gt_b = F.max_pool2d(1 - one_hot_gt, kernel_size=self.theta0, stride=1, padding=(self.theta0 - 1) // 2)
        gt_b -= 1 - one_hot_gt

        pred_b = F.max_pool2d(1 - pred, kernel_size=self.theta0, stride=1, padding=(self.theta0 - 1) // 2)
        pred_b -= 1 - pred

        # extended boundary map
        gt_b_ext = F.max_pool2d(gt_b, kernel_size=self.theta, stride=1, padding=(self.theta - 1) // 2)
        pred_b_ext = F.max_pool2d(pred_b, kernel_size=self.theta, stride=1, padding=(self.theta - 1) // 2)
     
        # reshape
        gt_b = gt_b.view(n, c, -1)
        pred_b = pred_b.view(n, c, -1)
        gt_b_ext = gt_b_ext.view(n, c, -1)
        pred_b_ext = pred_b_ext.view(n, c, -1)

        # Precision, Recall
        P = torch.sum(pred_b * gt_b_ext, dim=2) / (torch.sum(pred_b, dim=2) + 1e-7)
        R = torch.sum(pred_b_ext * gt_b, dim=2) / (torch.sum(gt_b, dim=2) + 1e-7)

        # Boundary F1 Score
        BF1 = 2 * P * R / (P + R + 1e-7)

        # summing BF1 Score for each class and average over mini-batch
        loss = torch.mean(1 - BF1)

        return loss    

# class BoundaryLoss(nn.Module):
#     """Boundary Loss proposed in:
#     Alexey Bokhovkin et al., Boundary Loss for Remote Sensing Imagery Semantic Segmentation
#     https://arxiv.org/abs/1905.07852
#     """

#     def __init__(self, theta0=3, theta=5, dims=4):
#         super().__init__()

#         self.theta0 = theta0
#         self.theta = theta
#         self.dims = dims

#     def forward(self, pred, gt):
#         """
#         Input:
#             - pred: the output from model (after softmax)
#                     shape (N, C, H, W)
#             - gt: ground truth map , float type
#                     shape (N, C, H, w)
#         Return:
#             - boundary loss, averaged over mini-batch
#         """
# #         n, c, _, _ = pred.shape
#         n = pred.shape[0]
#         c = pred.shape[1]

#         # one-hot vector of ground truth
#         one_hot_gt = gt #one_hot(gt, c)

#         # boundary map
#         gt_b = F.max_pool2d(1 - one_hot_gt, kernel_size=self.theta0, stride=1, padding=(self.theta0 - 1) // 2)
#         gt_b -= 1 - one_hot_gt

#         pred_b = F.max_pool2d(1 - pred, kernel_size=self.theta0, stride=1, padding=(self.theta0 - 1) // 2)
#         pred_b -= 1 - pred

#         # extended boundary map
#         gt_b_ext = F.max_pool2d(gt_b, kernel_size=self.theta, stride=1, padding=(self.theta - 1) // 2)
#         pred_b_ext = F.max_pool2d(pred_b, kernel_size=self.theta, stride=1, padding=(self.theta - 1) // 2)
     
#         # reshape
#         gt_b = gt_b.view(n, c, -1)
#         pred_b = pred_b.view(n, c, -1)
#         gt_b_ext = gt_b_ext.view(n, c, -1)
#         pred_b_ext = pred_b_ext.view(n, c, -1)

#         # Precision, Recall
#         P = torch.sum(pred_b * gt_b_ext, dim=2) / (torch.sum(pred_b, dim=2) + 1e-7)
#         R = torch.sum(pred_b_ext * gt_b, dim=2) / (torch.sum(gt_b, dim=2) + 1e-7)
        
#         TP = torch.sum(pred_b * gt_b_ext, dim=2)
#         FP = torch.sum(pred_b * gt_b, dim=2) 
#         FN = 
#         alpha= 0.7
        
#         # Boundary F1 Score
#         BF1 = 2 * P * R / (P + R + 1e-7)

#         # summing BF1 Score for each class and average over mini-batch
#         loss = torch.mean(1 - BF1)

#         return loss    
    
import numpy as np
import cv2
import torch
import torchvision

# def opencv_skelitonize(img):
#     skel = np.zeros(img.shape, np.uint8)
#     img = img.astype(np.uint8)
#     size = np.size(img)
#     element = cv2.getStructuringElement(cv2.MORPH_CROSS,(3,3))
#     done = False
#     while( not done):
#         eroded = cv2.erode(img,element)
#         temp = cv2.dilate(eroded,element)
#         temp = cv2.subtract(img,temp)
#         skel = cv2.bitwise_or(skel,temp)
#         img = eroded.copy()
#         zeros = size - cv2.countNonZero(img)
#         if zeros==size:
#             done = True
#     return skel

def dice_loss(pred, target):
    '''
    inputs shape  (batch, channel, height, width).
    calculate dice loss per batch and channel of sample.
    E.g. if batch shape is [64, 1, 128, 128] -> [64, 1]
    '''
    smooth = 1.
    N = pred.shape[0]
    C = pred.shape[1]
    iflat = pred.view(N,C, -1) #batch, channel, -1
    tflat = target.view(N,C, -1)
    intersection = (iflat * tflat).sum(-1)
    return -((2. * intersection + smooth) / (iflat.sum(-1) + tflat.sum(-1) + smooth))

# def soft_skeletonize(x, thresh_width=10):
# # def soft_skeletonize(x, thresh_width=3):
#     '''
#     Differenciable aproximation of morphological skelitonization operaton
#     thresh_width - maximal expected width of vessel
#     '''
#     for i in range(thresh_width):
#         min_pool_x = torch.nn.functional.max_pool2d(x*-1, (3, 3), 1, 1)*-1
#         contour = torch.nn.functional.relu(torch.nn.functional.max_pool2d(min_pool_x, (3, 3), 1, 1) - min_pool_x)
#         x = torch.nn.functional.relu(x - contour)
#     return x

def soft_skeletonize(x, thresh_width = 20):
    '''
    Differenciable aproximation of morphological skelitonization operaton
    thresh_width - maximal expected width of vessel
    '''
    for i in range(thresh_width):
        min_pool_x = torch.nn.functional.max_pool2d(x*-1, (3, 3), 1, 1)*-1
        contour = torch.nn.functional.relu(torch.nn.functional.max_pool2d(min_pool_x, (3, 3), 1, 1) - min_pool_x)
        x = torch.nn.functional.relu(x - contour)
    return x

# def norm_intersection(center_line, vessel):
#     '''
#     inputs shape  (batch, channel, height, width)
#     intersection formalized by first ares
#     x - suppose to be centerline of vessel (pred or gt) and y - is vessel (pred or gt)
#     '''
#     smooth = 1.
#     clf = center_line.view(*center_line.shape[:2], -1)
#     vf = vessel.view(*vessel.shape[:2], -1)
#     intersection = (clf * vf).sum(-1)
#     return (intersection + smooth) / (clf.sum(-1) + smooth)

from skimage.morphology import skeletonize

# def soft_cldice_loss(pred, target, target_skeleton=None):
#     '''
#     inputs shape  (batch, channel, height, width).
#     '''
#     pred_skeleton = soft_skeletonize(pred)
#     if target_skeleton is None:
#         target_skeleton = soft_skeletonize(target)
        
# #     cl_pred = skeletons(pred)
# #     if target_skeleton is None:
# #         target_skeleton = skeletons(target)    
        
# #     pred_skeleton = skeletons(pred)
# #     target_skeleton = skeletons(target)
    
#     plt.figure(figsize=(20,20))
#     plt.subplot(121)
#     plt.imshow(pred_skeleton[0,0])
#     plt.subplot(122)
#     plt.imshow(target_skeleton[0,0])
        
#     TP = torch.sum(pred_skeleton*target_skeleton)
#     FP = torch.sum(pred_skeleton*(1-target_skeleton))
#     FN = torch.sum((1-pred_skeleton)*target_skeleton)
    
#     return 1 -((2. * TP) / (2*TP + FP + FN))

# def soft_cldice_loss(pred, target, target_skeleton=None):
#     '''
#     inputs shape  (batch, channel, height, width).
#     '''
# #     cl_pred = soft_skeletonize(pred)
# #     if target_skeleton is None:
# #         target_skeleton = soft_skeletonize(target)
        
# #     cl_pred = skeletons(pred)
# #     if target_skeleton is None:
# #         target_skeleton = skeletons(target)    
        
#     pred_skeleton = skeletons(pred)
#     target_skeleton = skeletons(target)
    
#     plt.figure(figsize=(20,20))
#     plt.subplot(121)
#     plt.subplot(122)
#     plt.imshow(target_skeleton[0,0])
        
#     TP = torch.sum(pred_skeleton*target_skeleton)
#     FP = torch.sum(pred_skeleton*(1-target_skeleton))
#     FN = torch.sum((1-pred_skeleton)*target_skeleton)
    
#     return 1 -((2. * TP) / (2*TP + FP + FN))

def soft_cldice_loss(pred, target, target_skeleton=None):
    '''
    inputs shape  (batch, channel, height, width).
    '''
#     cl_pred = soft_skeletonize(pred)
#     if target_skeleton is None:
#         target_skeleton = soft_skeletonize(target)
        
#     cl_pred = skeletons(pred)
#     if target_skeleton is None:
#         target_skeleton = skeletons(target)    
        
#     pred_skeleton = skeletons(pred.cpu().detach().numpy()).to(device)
#     target_skeleton = skeletons(target.cpu().detach().numpy()).to(device)
        
    pred_skeleton = skeletons(pred.cpu().detach().numpy())#.to(device)
    target_skeleton = skeletons(target.cpu().detach().numpy())#.to(device)
    
#     plt.figure(figsize=(20,20))
#     plt.subplot(121)
#     plt.imshow(pred_skeleton[0,0])
#     plt.subplot(122)
#     plt.imshow(target_skeleton[0,0])
        
    TP = torch.sum(pred_skeleton*target_skeleton)
    FP = torch.sum(pred_skeleton*(1-target_skeleton))
    FN = torch.sum((1-pred_skeleton)*target_skeleton)
    
    return 1 -((2. * TP + 1) / (2*TP + FP + FN + 1))

# def soft_cldice_loss(pred, target, target_skeleton=None):
#     '''
#     inputs shape  (batch, channel, height, width).
#     calculate clDice loss
#     Because pred and target at moment of loss calculation will be a torch tensors
#     it is preferable to calculate target_skeleton on the step of batch forming,
#     when it will be in numpy array format by means of opencv
#     '''
# #     cl_pred = soft_skeletonize(pred)
# #     if target_skeleton is None:
# #         target_skeleton = soft_skeletonize(target)
        
# #     cl_pred = skeletons(pred)
# #     if target_skeleton is None:
# #         target_skeleton = skeletons(target)    
        
#     cl_pred = skeletons(pred.cpu().detach().numpy()).to(device)
#     target_skeleton = skeletons(target.cpu().detach().numpy()).to(device)
    
#     plt.figure(figsize=(20,20))
#     plt.subplot(121)
#     plt.imshow(cl_pred[0,0])
#     plt.subplot(122)
#     plt.imshow(target_skeleton[0,0])
        
#     iflat = norm_intersection(cl_pred, target)
#     tflat = norm_intersection(target_skeleton, pred)
#     intersection = iflat * tflat
#     return -((2. * intersection) / (iflat + tflat))

class clDiceLoss(nn.Module):

    def __init__(self):
        super().__init__()

    def forward(self, pred, gt):
        return soft_cldice_loss(pred,gt).squeeze()

def skeletons(mask):
    
    '''
    mask shape
    batch,channel,height,width    
    '''
    
    shape = mask.shape
    result = np.zeros_like(mask)
    
    for idx in range(shape[0]):
        result[idx,0] = dilation(skeletonize(mask[idx,0]),square(2))
        # 클러스터 없애기...
    return torch.tensor(result)

In [34]:
# !pip install monai
from monai.losses import DiceLoss
from monai.losses import GeneralizedDiceLoss

In [35]:
class compoundloss(nn.Module):
    """
    TODO : activation
    
    """
    def __init__(self, list_loss, activation=None, weight=None):
        super().__init__()
        self.weight=weight
        self.list_loss=list_loss
            
    def forward(self, pr, gt):
        
        if self.weight == None:
            self.weight = torch.ones(len(self.list_loss))/len(self.list_loss)
            
        lossfn = []
        for loss in self.list_loss:
            temp = loss
            lossfn.append(temp)
            
        loss = 0
        
        for idx in range(len(lossfn)):
            loss += lossfn[idx](pr,gt) * self.weight[idx]

        return torch.mean(loss)
    
# loss = compoundloss([BoundaryLoss(),clDiceLoss(),Mytorch.utils.losses.DiceLoss()],[.33,.33,.33])
loss = compoundloss([BoundaryLoss(),GeneralizedDiceLoss()],[.7,.3])

In [36]:
# !pip install git+https://github.com/qubvel/segmentation_models.pytorch
import segmentation_models_pytorch as smp
# model = smp.Unet('timm-efficientnet-b5', classes=1, activation='sigmoid')
# net = smp.Unet('timm-efficientnet-b7', classes=1, activation='sigmoid')
net = smp.FPN('timm-efficientnet-b7',encoder_weights=None, classes=1, activation='sigmoid')
# net = smp.FPN('timm-efficientnet-b7', classes=1, activation='sigmoid')
# net = smp.DeepLabV3Plus('timm-efficientnet-b5',encoder_weights=None, in_channels=3, classes=1, activation='sigmoid')
# net = smp.DeepLabV3Plus('timm-efficientnet-b5', in_channels=3, classes=1, activation='sigmoid')

In [37]:
def experiment_name():
    exam_name = net.__repr__().split('(')[0]+ \
                '_'+'Loss'+str(loss.list_loss)        
    return exam_name 

exam_name =  experiment_name()
# !pip install easydict
import easydict

settings = easydict.EasyDict({
                              "project_name":"Retina", 
                              "exam_name": exam_name,
                              "auto_scale_batch_size":'power',
                              "use_amp":'native',
                              "gpus": gpus,
                              'loss':loss,
})

hparams = easydict.EasyDict({
                          "epochs": 200,
                          "grad_batches":1,
                          "resume": False,
                          "batch_size":64,
                          "lr":0.001,
})

print('settings\n',settings)
print('hyperparams\n',hparams)

lr = 0.001
batch_size = 32
model = SegModel(**vars(hparams))

# logger = WandbLogger(project=hparams.project_name, name=hparams.exam_name)
# logger.watch(model.net)

settings
 {'project_name': 'Retina', 'exam_name': 'FPN_Loss[BoundaryLoss(), GeneralizedDiceLoss()]', 'auto_scale_batch_size': 'power', 'use_amp': 'native', 'gpus': '0,1,2,3', 'loss': compoundloss()}
hyperparams
 {'epochs': 200, 'grad_batches': 1, 'resume': False, 'batch_size': 64, 'lr': 0.001}


In [ ]:
import os

import torch
from torch.nn import functional as F
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST
from torchvision import transforms

import pytorch_lightning as pl
import neptune

CHECKPOINTS_DIR = 'mymodel/'

from pytorch_lightning.loggers.neptune import NeptuneLogger

neptune_logger = NeptuneLogger(
    api_key="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vdWkubmVwdHVuZS5haSIsImFwaV91cmwiOiJodHRwczovL3VpLm5lcHR1bmUuYWkiLCJhcGlfa2V5IjoiZGUwYmNlMTAtYzlmYy00MzAxLTk0MGItM2NmZjUxZGRkZWUzIn0=",
    project_name="kevinkwshin/test",
    close_after_fit=True,
    experiment_name='logs/'+exam_name,  
    params=hparams,
    
#     upload_source_files=['final_FPN.ipynb']
#     params={"max_epochs": MAX_EPOCHS,
#             "batch_size": BATCHSIZE,
#             "lr": LR},
)

from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint,EarlyStopping


model_checkpoint = pl.callbacks.ModelCheckpoint(filepath=CHECKPOINTS_DIR)

# checkpoint_callback = ModelCheckpoint(
#     filepath='mymodel/{epoch}-{val_loss:.2f}-{other_metric:.2f}'
# )
# early_stop_callback = EarlyStopping(monitor='val_loss', verbose=True,)

resume_from_checkpoint = 'best.ckpt'

trainer = Trainer(max_epochs=200,
                  logger=neptune_logger,
                  checkpoint_callback=model_checkpoint,
                  )
trainer.fit(model)
# trainer.test(model)

In [38]:
def pred_tta_seg(model,image,inference_size=512,tta_time = 100):
    
    B,C,image_h,image_w = image.shape # (B,C,H,W)
    pad_h = inference_size 
    pad_w = inference_size
    
    result = (torch.ones(1,1,image_h,image_w)/2)
    
    h_delta = inference_size//2
    w_delta = inference_size//2
    
    for idx in range(tta_time):

        h_center = np.random.randint(h_delta,image_h-h_delta)
        w_center = np.random.randint(w_delta,image_w-w_delta)

        h1 = - h_delta + h_center
        h2 =   h_delta + h_center

        w1 = - w_delta + w_center
        w2 =   w_delta + w_center
    
        image_temp = image[:,:,h1:h2,w1:w2]
        result_temp = result[:,:,h1:h2,w1:w2]
        
        with torch.no_grad():
            pred = model(image_temp)
            result_temp = (result_temp+pred)/2
            result[:,:,h1:h2,w1:w2] = result_temp
    return result

In [39]:
from pytorch_lightning import Trainer

resume_from_checkpoint = 'mymodel/epoch=73.ckpt'

trainer = Trainer(max_epochs=1,
                  resume_from_checkpoint=resume_from_checkpoint,
#                   gpus=gpus,
                  )

trainer.test(model)

GPU available: True, used: False
TPU available: False, using: 0 TPU cores


Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.


torch.Size([1, 1, 1024, 1024]) [1.5618488e-18 1.5932064e-18 2.0240830e-18 ... 9.9999988e-01 9.9999994e-01
 1.0000000e+00]
tensor(0.4970)


Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.


torch.Size([1, 1, 1024, 1024]) [3.3617570e-20 4.5972588e-20 6.4369069e-20 ... 9.9999988e-01 9.9999994e-01
 1.0000000e+00]
tensor(0.4966)


...

Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.


torch.Size([1, 1, 1024, 1024]) [4.6391939e-19 5.7068690e-19 7.8449453e-19 ... 9.9999988e-01 9.9999994e-01
 1.0000000e+00]
tensor(0.4970)


Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.


torch.Size([1, 1, 1024, 1024]) [6.2351445e-21 6.7112927e-21 1.6811990e-20 ... 9.9999988e-01 9.9999994e-01
 1.0000000e+00]
tensor(0.4969)
--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_acc': tensor(0.4967), 'test_loss': tensor(0.5000)}
--------------------------------------------------------------------------------



[{'test_loss': 0.5, 'test_acc': 0.4967293441295624}]

In [ ]:
from monai.inferers import SlidingWindowInferer

class ToyModel:
    # A simple model generates the output by adding an integer `pred` to input.
    # each call of this instance increases the integer by 1.
    pred = 0
    def __call__(self, input):
        self.pred = self.pred + 1
        input = input + self.pred
        return input

infer = SlidingWindowInferer(roi_size=(512, 512), sw_batch_size=1, overlap=0.75)
input_tensor = torch.zeros(1, 3, 1024,1024)
output_tensor = infer(input_tensor, ToyModel())

In [ ]:
plt.imshow(output_tensor[0, 0]); plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(16, 12))
a = np.random.rand(512,512,3)
ax.histmshow(a)
neptune_logger.experiment.log_image(epoch, fig)

# Get predictions on external test

import numpy as np

# y_true, y_pred = [],[]
# for i, (x, y) in enumerate(test_loader):
#     y_hat = model.forward(x).argmax(axis=1).cpu().detach().numpy()
#     y = y.cpu().detach().numpy()

#     y_true.append(y)
#     y_pred.append(y_hat)

#     if i == len(test_loader):
#         break
# y_true = np.hstack(y_true)
# y_pred = np.hstack(y_pred)

# # Log additional metrics
# from sklearn.metrics import accuracy_score

# accuracy = accuracy_score(y_true, y_pred)
# neptune_logger.experiment.log_metric('test_accuracy', accuracy)

# # Log charts
# from scikitplot.metrics import plot_confusion_matrix
# import matplotlib.pyplot as plt

# fig, ax = plt.subplots(figsize=(16, 12))
# plot_confusion_matrix(y_true, y_pred, ax=ax)
# neptune_logger.experiment.log_image('confusion_matrix', fig)

# # Save checkpoints folder
# neptune_logger.experiment.log_artifact(CHECKPOINTS_DIR)

# # You can stop the experiment
# neptune_logger.experiment.stop()